# Swept Langmuir Analysis: Finding Peak Slope

This notebook covers the use of the [**find_didv_peak_location()**](../../../api/plasmapy.analysis.swept_langmuir.plasma_potential.find_didv_peak_location.rst#find-didv-peak-location) function which finds the bias location associated with the peak slope of the swept Langmuir traces.

The bias voltage at the peaks slope is often used as a rough estimate for the plasma potential, but will always be slighlty lower than the actual plasma potential.

In [ ]:
%matplotlib inline

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

from plasmapy.analysis import swept_langmuir as sla

plt.rcParams["figure.figsize"] = [10.5, 0.56 * 10.5]

## Contents:

1. [How find_didv_peak_location() works](#How-find_didv_peak_location()-works)
    1. [Notes about usage](#Notes-about-usage)
    1. [Knobs to turn](#Knobs-to-turn)
1. [Calculate the Floating Potential](#Calculate-the-Floating-Potential)
    1. [Interpreting results](#Interpreting-results)
    1. [Plotting results](#Plotting-results)

## How `find_didv_peak_location()` works

## Calcualte the Peak Slope

In [ ]:
# load data
filename = "Beckers2017_noisy.npy"
# filename = "Pace2015.npy"
filepath = (Path.cwd() / ".." / ".." / "langmuir_samples" / filename).resolve()
voltage, current = np.load(filepath)

# voltage array needs to be monotonically increasing/decreasing
voltage, current = sla.sort_sweep_arrays(voltage, current)

# grabbing floating potential for future use
vf, _ = sla.find_floating_potential(voltage, current)
print(f"floating potential = {vf:.2f} V")

### Interpreting results

In [ ]:
results = sla.find_didv_peak_location(voltage, current, voltage_window=[vf, None])

# bias at peak slope
results[0]

In [ ]:
results[1].data_slice

In [ ]:
np.where(voltage >= -30)[0][0]

### Plotting results

In [ ]:
figwidth, figheight = plt.rcParams["figure.figsize"]
figheight = 1.5 * figheight
fig, axs = plt.subplots(2, 1, figsize=[figwidth, figheight])
_font_size = 12

bias = results[0]
extras = results[1]

# first (I vs. V) plot
axs[0].set_xlabel("Bias Voltage (V)", fontsize=_font_size)
axs[0].set_ylabel("Current (A)", fontsize=_font_size)

axs[0].plot(voltage, current,  label="Sweep Data")

ylim = axs[0].get_ylim()
xlim = axs[0].get_xlim()
axs[0].set_ylim(ylim)
axs[0].axvline(bias, color="grey")
axs[0].fill_between(
    [bias - extras.std, bias + extras.std],
    2*[ylim[0]],
    2*[ylim[1]],
    color="grey",
    alpha=0.2,
)

txt = f"${bias:.2f} \\pm {extras.std:.2f}$ V\n"
txt_loc = [bias, ylim[1]]
txt_loc = axs[0].transData.transform(txt_loc)
txt_loc = axs[0].transAxes.inverted().transform(txt_loc)
txt_loc[0] -= 0.02
txt_loc[1] -= 0.15
axs[0].text(
    txt_loc[0],
    txt_loc[1],
    txt,
    fontsize="large",
    transform=axs[0].transAxes,
    ha="right",
)

# second (dI/dV vs V) plot
axs[1].set_xlabel("Bias Voltage (V)", fontsize=_font_size)
axs[1].set_ylabel("$dI/dV$ (A / V)", fontsize=_font_size)

axs[1].set_xlim(xlim)

voltage_merge, current_merge = sla.merge_voltage_clusters(voltage, current, voltage_step_size=0)
voltage_slice = voltage_merge[extras.data_slice]
current_slice = current_merge[extras.data_slice]

_colors = plt.get_cmap("OrRd")(np.linspace(0.2, 0.8, len(extras.savgol_windows)))
for ii, _window in enumerate(extras.savgol_windows):
    v_smooth = signal.savgol_filter(voltage_slice, _window, 1)
    c_smooth = signal.savgol_filter(current_slice, _window, 1)

    didv = np.gradient(c_smooth, v_smooth)

    axs[1].plot(
        v_smooth[_window:-_window],
        didv[_window:-_window],
        color=_colors[ii],
        label=f"Savgol window {_window}",
    )

ylim = axs[1].get_ylim()
axs[1].set_ylim(ylim)

axs[1].axvline(bias, color="grey", zorder=100)
axs[1].fill_between(
    [bias - extras.std, bias + extras.std],
    2*[ylim[0]],
    2*[ylim[1]],
    color="grey",
    alpha=0.2,
    zorder=0
)

axs[1].legend();